In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import numpy as np
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier
from emoticons import (positive_emoticons, negative_emoticons, positive_sentiment, negative_sentiment)
from nltk.tokenize import word_tokenize
import string
from operator import itemgetter

In [2]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/cassiobotaro/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/cassiobotaro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
classificator = SVC(kernel='linear')

training_set = ['o cruzeiro jogou muito bem', 'parabéns pela vitória cruzeiro',
                'ainda acho o cruzeiro ruim', 'o cruzeiro não jogou bem',
                'cruzeiro jogou contra o são paulo',
                'primeira rodada o cruzeiro enfrentou o sport']
labels = [1, 1, -1, -1, 0, 0]

tf_vectorizer = TfidfVectorizer(stop_words=stopwords.words('portuguese'),
                                analyzer='word', ngram_range=(1, 1),
                                lowercase=True, use_idf=True,
                                strip_accents='unicode')

features = tf_vectorizer.fit_transform(training_set)
classificator.fit(features, labels)

tweets = ['cruzeiro tem um bom time', 'acompanhe os jogos da primeira rodada',
          'muito ruim esse jogo do cruzeiro']

vector_tweets = tf_vectorizer.transform(tweets)
predictions = classificator.predict(vector_tweets)

for prediction, tweet in zip(predictions, tweets):
    print(f'{tweet} -> {prediction}')


cruzeiro tem um bom time -> 1
acompanhe os jogos da primeira rodada -> 0
muito ruim esse jogo do cruzeiro -> -1


/home/cassiobotaro/.virtualenvs/sentibol/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ate', 'eramos', 'estao', 'estavamos', 'estiveramos', 'estivessemos', 'foramos', 'fossemos', 'ha', 'hao', 'houveramos', 'houverao', 'houveriamos', 'houvessemos', 'ja', 'nao', 'sao', 'sera', 'serao', 'seriamos', 'so', 'tambem', 'tera', 'terao', 'teriamos', 'tinhamos', 'tiveramos', 'tivessemos', 'voce', 'voces'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [4]:
classificator = MultinomialNB(alpha=.01)

training_set = ['o cruzeiro jogou muito bem', 'parabéns pela vitória cruzeiro',
                'ainda acho o cruzeiro ruim', 'o cruzeiro não jogou bem',
                'cruzeiro jogou contra o sport',
                'primeira rodada o cruzeiro enfrentou o sport']
labels = [1, 1, -1, -1, 0, 0]

tf_vectorizer = TfidfVectorizer(stop_words=stopwords.words('portuguese'),
                                analyzer='word', ngram_range=(1, 1),
                                lowercase=True, use_idf=True,
                                strip_accents='unicode')

features = tf_vectorizer.fit_transform(training_set)
classificator.fit(features, labels)

tweets = ['cruzeiro tem um bom time', 'acompanhe os jogos da primeira rodada',
          'muito ruim esse jogo do cruzeiro']

vector_tweets = tf_vectorizer.transform(tweets)
predictions = classificator.predict(vector_tweets)
for prediction, tweet in zip(predictions, tweets):
    print(f'{tweet} -> {prediction}')


cruzeiro tem um bom time -> 1
acompanhe os jogos da primeira rodada -> 0
muito ruim esse jogo do cruzeiro -> -1


In [5]:
# lists based on https://en.wikipedia.org/wiki/List_of_emoticons
# Versão original removia pontuação(o que é errado, pois emoticons são feitos utilizando pontuação)
# falta de emojis
# deveria trazer as palavras a sua raiz

list_emoticons_positive = positive_emoticons.split('\n')
list_emoticons_negative = negative_emoticons.split('\n')
list_negative = negative_sentiment.split('\n')
list_positive = positive_sentiment.split('\n')


def classify(text):
    for t in text.split(' '):
        if t in list_emoticons_positive:
            return 1
        elif t in list_emoticons_negative:
            return -1
    return votation(text)


def votation(text):
    vote = 0
    text = clean_text(text)

    for t in text:
        if t in list_negative:
            vote = vote - 1
        elif t in list_positive:
            vote = vote + 1

    if vote > 1:
        return 1
    elif vote < 0:
        return -1
    else:
        return vote


def clean_text(text):
    token_text = word_tokenize(text.lower())
    text = (word for word in token_text
            if word not in stopwords.words('portuguese'))
    return text


def predict(tweets):
    return np.array([classify(phrase) for phrase in tweets])


tweets = ['cruzeiro tem um bom time', 'acompanhe os jogos da primeira rodada',
          'muito ruim esse jogo do cruzeiro', 'cruzeiro! :)',
          'Jogo morno entre cruzeiro e são paulo',
          'Meu sport perdeu hoje :\'(']

predictions = predict(tweets)
for prediction, tweet in zip(predictions, tweets):
    print(f'{tweet} -> {prediction}')


cruzeiro tem um bom time -> 1
acompanhe os jogos da primeira rodada -> 0
muito ruim esse jogo do cruzeiro -> -1
cruzeiro! :) -> 1
Jogo morno entre cruzeiro e são paulo -> 0
Meu sport perdeu hoje :'( -> -1


In [6]:
classificator = VotingClassifier(estimators=[
    ('svm', SVC(kernel='linear')),
    ('naive', MultinomialNB(alpha=.01))]
)

training_set = ['o cruzeiro jogou muito bem', 'parabéns pela vitória cruzeiro',
                'ainda acho o cruzeiro ruim', 'o cruzeiro não jogou bem',
                'cruzeiro jogou contra o sport',
                'primeira rodada o cruzeiro enfrentou o sport']
labels = [1, 1, -1, -1, 0, 0]

tf_vectorizer = TfidfVectorizer(stop_words=stopwords.words('portuguese'),
                                analyzer='word', ngram_range=(1, 1),
                                lowercase=True, use_idf=True,
                                strip_accents='unicode')

features = tf_vectorizer.fit_transform(training_set)
classificator.fit(features, labels)
tweets = ['cruzeiro tem um bom time', 'acompanhe os jogos da primeira rodada',
          'muito ruim esse jogo do cruzeiro']
vector_tweets = tf_vectorizer.transform(tweets)
predictions = classificator.predict(vector_tweets)
for prediction, tweet in zip(predictions, tweets):
    print(f'{tweet} -> {prediction}')

cruzeiro tem um bom time -> 1
acompanhe os jogos da primeira rodada -> 0
muito ruim esse jogo do cruzeiro -> -1


/home/cassiobotaro/.virtualenvs/sentibol/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ate', 'eramos', 'estao', 'estavamos', 'estiveramos', 'estivessemos', 'foramos', 'fossemos', 'ha', 'hao', 'houveramos', 'houverao', 'houveriamos', 'houvessemos', 'ja', 'nao', 'sao', 'sera', 'serao', 'seriamos', 'so', 'tambem', 'tera', 'terao', 'teriamos', 'tinhamos', 'tiveramos', 'tivessemos', 'voce', 'voces'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [7]:


def compare_tweet_jaccard(tweet, tweet_compare):
    words_tweet = tweet.lower().split()
    words_compare = tweet_compare.lower().split()
    intersection = set(words_tweet).intersection(set(words_compare))
    intersection_size = len(intersection)
    calc = float(intersection_size) / (len(words_tweet) + len(words_compare)
                                       - intersection_size)

    if calc >= 0.20:  # Define o quao semelhante é um tweet com o outro
        return True  # se tweet igual a tweet_compare
    return False


tweets = ['o cruzeiro jogou muito bem', 'parabéns pela vitória cruzeiro',
          'ainda acho o cruzeiro ruim', 'o cruzeiro não jogou bem',
          'cruzeiro jogou contra o sport',
          'primeira rodada o cruzeiro enfrentou o sport',
          'bela vitória do cruzeiro', 'mais uma vitória do cruzeiro']

tf_vectorizer = TfidfVectorizer(
            stop_words=stopwords.words('portuguese'), analyzer='word',
            ngram_range=(1, 1), lowercase=True, use_idf=True)
matrix = tf_vectorizer.fit_transform(tweets)

feature_array = np.array(tf_vectorizer.get_feature_names())
tfidf_sorting = np.argsort(matrix.toarray()).flatten()[::-1]
feature_words = feature_array[tfidf_sorting][:10]
featured_tweets = []
for tweet in tweets:
    relevance = 0
    for word in feature_words:
        if word in tweet:
            relevance += 1
    if relevance > 3:
        featured_tweets.append((tweet, relevance))

    print(f'{tweet} -> {relevance}')

# eliminate duplicated tweets
non_duplicated = []
for t in sorted(featured_tweets, key=itemgetter(1), reverse=True):
    if not non_duplicated:
        non_duplicated.append(t[0])
    else:
        for nd in non_duplicated:
            if not compare_tweet_jaccard(nd, t[0]):
                non_duplicated.append(t[0])


print(80 * '-')
print('Tweets mais relevantes:')
print(non_duplicated)

o cruzeiro jogou muito bem -> 2
parabéns pela vitória cruzeiro -> 3
ainda acho o cruzeiro ruim -> 2
o cruzeiro não jogou bem -> 2
cruzeiro jogou contra o sport -> 4
primeira rodada o cruzeiro enfrentou o sport -> 5
bela vitória do cruzeiro -> 2
mais uma vitória do cruzeiro -> 2
--------------------------------------------------------------------------------
Tweets mais relevantes:
['primeira rodada o cruzeiro enfrentou o sport']
